## Binary Classification with different optimizers, schedulers, etc.

In this notebook we will use the Adult Census dataset. Download the data from [here](https://www.kaggle.com/wenruliu/adult-income-dataset/downloads/adult.csv/2).

In [1]:
import numpy as np
import pandas as pd
import torch

from pytorch_widedeep.preprocessing import WidePreprocessor, DensePreprocessor
from pytorch_widedeep.models import Wide, DeepDense, WideDeep
from pytorch_widedeep.metrics import BinaryAccuracy

In [2]:
df = pd.read_csv('data/adult/adult.csv.zip')
df.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
# For convenience, we'll replace '-' with '_'
df.columns = [c.replace("-", "_") for c in df.columns]
# binary target
df['income_label'] = (df["income"].apply(lambda x: ">50K" in x)).astype(int)
df.drop('income', axis=1, inplace=True)
df.head()

,age,workclass,fnlwgt,education,educational_num,marital_status,occupation,relationship,race,gender,capital_gain,capital_loss,hours_per_week,native_country,income_label
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,0
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,0
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,1
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,1
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,0


### Preparing the data

Have a look to notebooks one and two if you want to get a good understanding of the next few lines of code (although there is no need to use the package)

In [4]:
wide_cols = ['education', 'relationship','workclass','occupation','native_country','gender']
crossed_cols = [('education', 'occupation'), ('native_country', 'occupation')]
cat_embed_cols = [('education',16), ('relationship',8), ('workclass',16), ('occupation',16),('native_country',16)]
continuous_cols = ["age","hours_per_week"]
target_col = 'income_label'

In [9]:
# TARGET
target = df[target_col].values

# WIDE
preprocess_wide = WidePreprocessor(wide_cols=wide_cols, crossed_cols=crossed_cols)
X_wide = preprocess_wide.fit_transform(df)

# DEEP
preprocess_deep = DensePreprocessor(embed_cols=cat_embed_cols, continuous_cols=continuous_cols)
X_deep = preprocess_deep.fit_transform(df)

In [10]:
print(X_wide)
print(X_wide.shape)

[[0. 1. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]
(48842, 796)


In [11]:
print(X_deep)
print(X_deep.shape)

[[ 0.          0.          0.         ...  0.         -0.99512893
  -0.03408696]
 [ 1.          1.          0.         ...  0.         -0.04694151
   0.77292975]
 [ 2.          1.          1.         ...  0.         -0.77631645
  -0.03408696]
 ...
 [ 1.          3.          0.         ...  0.          1.41180837
  -0.03408696]
 [ 1.          0.          0.         ...  0.         -1.21394141
  -1.64812038]
 [ 1.          4.          6.         ...  0.          0.97418341
  -0.03408696]]
(48842, 7)


As you can see, you can run a wide and deep model in just a few lines of code

Let's now see how to use `WideDeep` with varying parameters

###  2.1 Dropout and Batchnorm

In [12]:
wide = Wide(wide_dim=X_wide.shape[1], pred_dim=1)
# We can add dropout and batchnorm to the dense layers
deepdense = DeepDense(hidden_layers=[64,32], dropout=[0.5, 0.5], batchnorm=True,
                      deep_column_idx=preprocess_deep.deep_column_idx,
                      embed_input=preprocess_deep.embeddings_input,
                      continuous_cols=continuous_cols)
model = WideDeep(wide=wide, deepdense=deepdense)

In [13]:
model

WideDeep(
  (wide): Wide(
    (wide_linear): Linear(in_features=796, out_features=1, bias=True)
  )
  (deepdense): Sequential(
    (0): DeepDense(
      (embed_layers): ModuleDict(
        (emb_layer_education): Embedding(17, 16)
        (emb_layer_native_country): Embedding(43, 16)
        (emb_layer_occupation): Embedding(16, 16)
        (emb_layer_relationship): Embedding(7, 8)
        (emb_layer_workclass): Embedding(10, 16)
      )
      (embed_dropout): Dropout(p=0.0, inplace=False)
      (dense): Sequential(
        (dense_layer_0): Sequential(
          (0): Linear(in_features=74, out_features=64, bias=True)
          (1): LeakyReLU(negative_slope=0.01, inplace=True)
          (2): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (3): Dropout(p=0.5, inplace=False)
        )
        (dense_layer_1): Sequential(
          (0): Linear(in_features=64, out_features=32, bias=True)
          (1): LeakyReLU(negative_slope=0.01, inplace=True)
   

We can use different initializers, optimizers and learning rate schedulers for each `branch` of the model

###  Optimizers, LR schedulers, Initializers and Callbacks

In [14]:
from pytorch_widedeep.initializers import KaimingNormal, XavierNormal
from pytorch_widedeep.callbacks import ModelCheckpoint, LRHistory, EarlyStopping
from pytorch_widedeep.optim import RAdam

In [15]:
# Optimizers
wide_opt = torch.optim.Adam(model.wide.parameters())
deep_opt = RAdam(model.deepdense.parameters())
# LR Schedulers
wide_sch = torch.optim.lr_scheduler.StepLR(wide_opt, step_size=3)
deep_sch = torch.optim.lr_scheduler.StepLR(deep_opt, step_size=5)

the component-dependent settings must be passed as dictionaries, while general settings are simply lists

In [16]:
# Component-dependent settings as Dict
optimizers = {'wide': wide_opt, 'deepdense':deep_opt}
schedulers = {'wide': wide_sch, 'deepdense':deep_sch}
initializers = {'wide': KaimingNormal, 'deepdense':XavierNormal}
# General settings as List
callbacks = [LRHistory(n_epochs=10), EarlyStopping, ModelCheckpoint(filepath='model_weights/wd_out')]
metrics = [BinaryAccuracy]

In [17]:
model.compile(method='binary', optimizers=optimizers, lr_schedulers=schedulers, 
              initializers=initializers,
              callbacks=callbacks,
              metrics=metrics)

In [18]:
model.fit(X_wide=X_wide, X_deep=X_deep, target=target, n_epochs=10, batch_size=256, val_split=0.2)

  0%|          | 0/153 [00:00<?, ?it/s]

Training


epoch 1:   0%|          | 0/153 [00:00<?, ?it/s]../torch/csrc/utils/python_arg_parser.cpp:756: UserWarning: This overload of addcmul_ is deprecated:
	addcmul_(Number value, Tensor tensor1, Tensor tensor2)
Consider using one of the following signatures instead:
	addcmul_(Tensor tensor1, Tensor tensor2, *, Number value)
valid: 100%|██████████| 39/39 [00:00<00:00, 111.64it/s, loss=0.351, metrics={'acc': 0.8226}]


In [19]:
dir(model)

['__call__',
 '__class__',
 '__delattr__',
 '__dict__',
 '__dir__',
 '__doc__',
 '__eq__',
 '__format__',
 '__ge__',
 '__getattr__',
 '__getattribute__',
 '__gt__',
 '__hash__',
 '__init__',
 '__init_subclass__',
 '__le__',
 '__lt__',
 '__module__',
 '__ne__',
 '__new__',
 '__reduce__',
 '__reduce_ex__',
 '__repr__',
 '__setattr__',
 '__setstate__',
 '__sizeof__',
 '__str__',
 '__subclasshook__',
 '__weakref__',
 '_activation_fn',
 '_apply',
 '_backward_hooks',
 '_buffers',
 '_forward_hooks',
 '_forward_pre_hooks',
 '_get_name',
 '_load_from_state_dict',
 '_load_state_dict_pre_hooks',
 '_loss_fn',
 '_lr_scheduler_step',
 '_modules',
 '_named_members',
 '_parameters',
 '_predict',
 '_register_load_state_dict_pre_hook',
 '_register_state_dict_hook',
 '_replicate_for_data_parallel',
 '_save_to_state_dict',
 '_slow_forward',
 '_state_dict_hooks',
 '_train_val_split',
 '_training_step',
 '_validation_step',
 '_version',
 '_warm_up',
 'add_module',
 'apply',
 'batch_size',
 'bfloat16',
 'buf

You see that, among many methods and attributes we have the `history` and `lr_history` attributes

In [20]:
model.history.epoch

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9]

In [21]:
print(model.history._history)

{'train_loss': [0.7607731277646582, 0.5239557282986984, 0.4440380584180745, 0.41758152647735247, 0.3964089655798245, 0.3891625396566453, 0.38853760053908903, 0.386707517835829, 0.38487764998199114, 0.3844185426733852], 'train_acc': [0.6362, 0.7566, 0.7964, 0.8055, 0.8152, 0.8158, 0.8159, 0.8164, 0.8172, 0.8192], 'val_loss': [0.40992657114297915, 0.37194269054975265, 0.3614824322553781, 0.3569044936925937, 0.3527434239020714, 0.3521107450509683, 0.35160034818527025, 0.35168310694205457, 0.35136762796304166, 0.3508173334292876], 'val_acc': [0.6714, 0.7719, 0.8042, 0.8116, 0.8195, 0.82, 0.82, 0.8202, 0.8209, 0.8226]}


In [22]:
print(model.lr_history)

{'lr_wide_0': [0.001, 0.001, 0.001, 0.0001, 0.0001, 0.0001, 1e-05, 1e-05, 1e-05, 1.0000000000000002e-06], 'lr_deepdense_0': [0.001, 0.001, 0.001, 0.001, 0.001, 0.0001, 0.0001, 0.0001, 0.0001, 0.0001]}


We can see that the learning rate effectively decreases by a factor of 0.1 (the default) after the corresponding `step_size`. Note that the keys of the dictionary have a suffix `_0`. This is because if you pass different parameter groups to the torch optimizers, these will also be recorded. We'll see this in the `Regression` notebook. 

And I guess one has a good idea of how to use the package. Before we leave this notebook just mentioning that the `WideDeep` class comes with a useful method to "rescue" the learned embeddings. For example, let's say I want to use the embeddings learned for the different levels of the categorical feature `education`

In [23]:
model.get_embeddings(col_name='education', cat_encoding_dict=preprocess_deep.label_encoder.encoding_dict)

{'11th': array([ 0.37296116,  0.05184834,  0.3131211 ,  0.22052889,  0.08505874,
        -0.07266734,  0.03647961, -0.02021132,  0.24249516, -0.1176727 ,
        -0.32592112,  0.08129914,  0.01013801,  0.01507128, -0.10127561,
        -0.6763426 ], dtype=float32),
 'HS-grad': array([ 0.34629953, -0.16661926, -0.02384926, -0.06235064,  0.32661676,
        -0.262116  ,  0.08339449,  0.17517047,  0.16401216,  0.0832316 ,
         0.00916898, -0.00509361, -0.51719534, -0.1961552 , -0.06048429,
         0.3759702 ], dtype=float32),
 'Assoc-acdm': array([ 0.16078217, -0.10902341,  0.09899426, -0.05065758,  0.21825325,
         0.13163188, -0.1288552 , -0.42406723, -0.11526844, -0.4670248 ,
        -0.10699925, -0.00966803,  0.07855067, -0.07996878, -0.09443142,
         0.40587   ], dtype=float32),
 'Some-college': array([-0.02036662, -0.31137288, -0.12254954, -0.14743474,  0.13482754,
         0.09543735, -0.07154643, -0.18334292,  0.36071104,  0.16267166,
        -0.19764908,  0.06264469, 